# Voter Disenfranchisement - Data Preparation

### Initial data exploration and clean up

We will be looking at how citizens of voting age can be, and are being disenfranchised in the United States of America from these three points of view - the citizens and groups that are being adversely affected by disenfranchisement (Who), how location plays into it the Where) and the temporal concerns that contribute to it (the When).

## 1. Electoral and temporal concerns contributing towards disenfranchisement

Our main data source for this project will be 2016 US Census data. We downloaded multiple files, each with various demographic information such as race, education and employment. We merged these together along with results from the 2016 election to use as our main data file. 

Information on mail-in and early voting was found online (http://www.ncsl.org/research/elections-and-campaigns/absentee-and-early-voting.aspx). We took this information and entered it into a .csv file to use in our analysis.

In [1]:
import pandas as pd

Load Labor, Race, and Income Demographics

In [4]:
labor = pd.read_csv("labor_demographics.csv")

In [6]:
race = pd.read_csv("ethnicity_income_demographics.csv")

Convert counts of ethnicity to rates.

In [27]:
race['Other'] = race['Other']/race['RACE-One-race']

In [28]:
race['Pacific Islander'] = race['Pacific Islander']/race['RACE-One-race']

In [29]:
race['Asian'] = race['Asian']/race['RACE-One-race']

In [30]:
race['Native American'] = race['Native American']/race['RACE-One-race']

In [31]:
race['African American'] = race['African American']/race['RACE-One-race']

In [35]:
race['White'] = race['White']/race['RACE-One-race']

Load Education Demographics

In [37]:
edu = pd.read_csv("education_demographics.csv")

2 Step Merge process to combine the 3 demographics files.

In [38]:
merge1 = labor.merge(race, left_on='Id2', right_on='Id2')

In [40]:
merge2 = merge1.merge(edu, left_on='Id2', right_on='Id2')

In [43]:
merge2.drop('Id_y', inplace = True, axis = 1)
merge2.drop('Geography_y', inplace = True, axis = 1)

Load Election Data by county from 2016.

In [20]:
election = pd.read_csv("election2016.csv")

In [82]:
election.tail()

,county,fips,cand,st,pct_report,votes,total_votes,lead,pct,state.name,Trump,Clinton,Other
18340,Niobrara County,56027,Hillary Clinton,WY,1.0,115,1297,Donald Trump,0.088666,Wyoming,0,115,0
18341,Niobrara County,56027,Gary Johnson,WY,1.0,49,1297,Donald Trump,0.037779,Wyoming,0,0,49
18342,Niobrara County,56027,Darrell Castle,WY,1.0,7,1297,Donald Trump,0.005397,Wyoming,0,0,7
18343,Niobrara County,56027,Jill Stein,WY,1.0,7,1297,Donald Trump,0.005397,Wyoming,0,0,7
18344,Niobrara County,56027,Rocky De La Fuente,WY,1.0,3,1297,Donald Trump,0.002313,Wyoming,0,0,3


Clean up by producing columns with vote totals for Trump, Clinton, and Others. Then group by county in order to have each row be a record of vote totals per county.

In [51]:
election['Trump'] = (election['cand'] == 'Donald Trump')*election['votes']

In [54]:
election['Clinton'] = (election['cand']=='Hillary Clinton')*election['votes']

In [53]:
election['Other'] = ((election['cand']!= 'Donald Trump')&(election['cand']!='Hillary Clinton'))*election['votes']

In [60]:
election_grp = election.groupby(['fips','county','st','state.name']).agg({'total_votes':'max', 
                                                           'Trump':'max', 'Clinton':'max', 'Other':'sum'}).reset_index()

In [85]:
election_grp.head(3)

,fips,county,st,state.name,Other,Clinton,Trump,total_votes
0,10001,Kent County,DE,Delaware,0.052752,0.44910,0.498148,74253
1,10003,New Castle County,DE,Delaware,0.049933,0.62304,0.327027,261468
2,10005,Sussex County,DE,Delaware,0.036649,0.37168,0.591670,105814


Create rates per candidate.

In [62]:
election_grp['Other'] = election_grp['Other']/election_grp['total_votes']

In [65]:
election_grp['Clinton'] = election_grp['Clinton']/election_grp['total_votes']

In [64]:
election_grp['Trump'] = election_grp['Trump']/election_grp['total_votes']

In [76]:
merge2['Id2'] = merge2['Id2'].apply(pd.to_numeric)

In [77]:
election_grp['fips'] = election_grp['fips'].apply(pd.to_numeric)

In [78]:
census_with_election = merge2.merge(election_grp, left_on = 'Id2', right_on = 'fips', how = 'outer')

In [80]:
census_with_election.to_csv('census_with_election_data.csv')

## 2. Citizens and groups affected by disenfranchisement

In order to collect comprehensivewe collected data from 4 different sources

In [2]:
felons_all = pd.read_csv("2016_felon_disenfranchisement_state.csv")
felons_black = pd.read_csv("2016_black_felon_disenfranchisement_state.csv")
felon_disenf_laws = pd.read_csv("Felon_Disenfranchisement_Laws_state.csv")
voterID_laws = pd.read_csv("VoterID_laws.csv")

In [5]:
felons_merge = felons_all.merge(felons_black, left_on='State', right_on='State')

In [10]:
felon_voter_laws = felon_disenf_laws.merge(voterID_laws, left_on='State', right_on='StateName')

In [17]:
felons_merge.head()

,State,Prison_x,Parole_x,Felony probation_x,Jail_x,Post-sentence_x,Total_x,VAP_x,% Disenfranchised_x,Prison_y,Parole_y,Felony probation_y,Jail_y,Post-sentence_y,Total_y,VAP_y,% Disenfranchised_y
0,Alabama,"30,585","6,580","15,626","1,578","231,896","286,266","3,755,483",7.62%,"17,775","3,957","7,740",823,"113,629","143,924","952,671",15.11%
1,Alaska,"5,497","2,035","6,900",7,NaN,"14,439","552,166",2.61%,519,211,718,2,NaN,"1,450","21,219",6.83%
2,Arizona,"44,509","7,241","51,362","1,341","116,717","221,170","5,205,215",4.25%,"5,879",952,"5,654",361,"12,645","25,492","214,412",11.89%
3,Arkansas,"19,224","21,811","24,695",975,NaN,"66,705","2,272,904",2.93%,"8,524","8,844","8,676",62,NaN,"26,106","333,472",7.83%
4,California,"136,302","86,254",NaN,NaN,NaN,"222,557","30,023,902",0.74%,"39,451","23,939",NaN,NaN,NaN,"63,390","1,858,353",3.41%


In [18]:
felon_voter_laws.head()

,State,abbreviation,Prison Only,Prison & Parole,"Prison, Parole & Probation","Prison, parole, probation, & post-sentence",StateName,StrictID_reqd,PhotoID_reqd,Non-PhotoID,No_ID
0,Alabama,AL,0,0,0,1,Alabama,0,1,0,0
1,Alaska,AK,0,0,1,0,Alaska,0,0,1,0
2,Arizona,AZ,0,0,0,1,Arizona,1,0,1,0
3,Arkansas,AR,0,0,1,0,Arkansas,0,1,0,0
4,California,CA,0,1,0,0,California,0,0,0,0
